# Seldon Deploy Advanced SDK Examples

In this workshop, we will showcase the functionality available via the [Seldon Deploy SDK](https://github.com/SeldonIO/seldon-deploy-sdk/tree/master/python).  
The SDK is auto-generated on top of the [Seldon Deploy REST API](https://deploy.seldon.io/en/latest/contents/product-tour/api/index.html).  

The SDK can be very useful to automate data science and deployment workflows.  Here are the steps we will walk through:
* Set up the SDK using the `client_credentials` flow
* Train and deploy a simple SKLearn model
* Play around with the metadata catalog

<!-- * Train and deploy a canary model
* View the change using GitOps
* View all Seldon Deployments
* Add model metadata
* Add an outlier detector
* Add a drift detector
* Run a batch job
* Get an input distribution
* Add an explainer and get an explanation
* List Kubernetes resources
* Remove all deployments and catalog entries
OTHER
* Create a new user and add to group/project-->



### Setup

Install the required libraries

In [ ]:
!pip install seldon-deploy-sdk
!pip install -U scikit-learn
!pip install xgboost
!pip install pyyaml

Set up the API config and authentication

__IMPORTANT:__ Don't forget to replace "XXXX" with your cluster IP below.

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient
from seldon_deploy_sdk.auth import OIDCAuthenticator

SD_IP = "XXXXX"

config = Configuration()
config.auth_method = "client_credentials" #  could also use auth_code or password_grant
config.host = f"https://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_server = f"https://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
# config.username = "admin@seldon.io"
# config.password = "12341234"
config.verify_ssl = False

# Authenticate against an OIDC provider
auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()
api_client = ApiClient(config)

Show the machine user that we are logged in as

In [ ]:
from seldon_deploy_sdk import EnvironmentApi

environment_api = EnvironmentApi(api_client)
environment_api.read_user()

### Train and deploy a simple SKLearn model

Train a simple SKLearn linear regression model

In [ ]:
# Import the required modules
from sklearn.linear_model import LinearRegression

# Define the input data
X = [[1, 1], [2, 2], [3, 3]]

# Define the output data
y = [1, 2, 3]

# Create a linear regression model
model = LinearRegression()

# Train the model on the data
model.fit(X, y)

# Use the model to make predictions
predictions = model.predict([[4, 4],[5,5]])
print(predictions)  # This should output [4]

Serialize and save the model locally using joblib

In [ ]:
import joblib

model_file_name = "model.joblib"
joblib.dump(model, model_file_name)

Save the model in a public Google Storage bucket

In [ ]:
!gsutil cp model.joblib gs://andrew-seldon/sdk-examples/simple-lr/model.joblib

##### Deploy this as a Core v1 Deployment

Create a SeldonDeployment as a YAML string (this is to show how SeldonDeployments are typically structured)

In [ ]:
DEPLOYMENT_NAME = "simple-lr"
NAMESPACE = "seldon-gitops"
URI = "gs://andrew-seldon/sdk-examples/simple-lr"

simple_lr_sdep = f"""
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: {DEPLOYMENT_NAME}
  namespace: {NAMESPACE}
spec:
  protocol: v2
  predictors:
    - name: default
      graph:
        name: {DEPLOYMENT_NAME}-model
        implementation: SKLEARN_SERVER
        modelUri: {URI}
"""

Load the YAML string in as a Python dictionary and deploy.

Create an instance of the SeldonDeploymentsApi and create a new deployment. 

In [ ]:
from seldon_deploy_sdk import SeldonDeploymentsApi
import yaml

mldeployment = yaml.safe_load(simple_lr_sdep)

deployment_api = SeldonDeploymentsApi(api_client)
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Check the status of the deployment

In [ ]:
status = ""
while status != "Available":
    new_status =  deployment_api.read_seldon_deployment(name=DEPLOYMENT_NAME, namespace=NAMESPACE).status.state
    if new_status != status: print(new_status)
    status = new_status

Build the endpoint for your deployed model

In [ ]:
endpoint = f"http://{SD_IP}/seldon/{NAMESPACE}/{DEPLOYMENT_NAME}/v2/models/{DEPLOYMENT_NAME}-model/infer"
endpoint

Send a simple test request with two inferences

In [ ]:
import requests
import json

request = {"inputs": [
    {"name": "input-1",
    "datatype": 
    "INT32",
    "shape": [2, 2],
    "data": [
        [4, 4],
        [7, 7]]
        }]}

response = requests.post(endpoint, json=request)
print(json.dumps(response.json(), indent=2))

##### Deploy as a Core v2 pipeline
First, we will need to push a model-settings.json file to the same model folder in Google Cloud Storage

In [ ]:
%%writefile model-settings.json
{
    "name": "simple-lr-model",
    "implementation": "mlserver_sklearn.SKLearnModel"
}

In [ ]:
!gsutil cp model-settings.json gs://andrew-seldon/sdk-examples/simple-lr/model-settings.json

In [ ]:
!gsutil ls gs://andrew-seldon/sdk-examples/simple-lr

Create the model 

In [ ]:
MODEL_NAME = "simple-lr-model2"

simple_lr_model = f"""
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: {MODEL_NAME}
  namespace: {NAMESPACE}
spec:
  storageUri: {URI}
  requirements:
  - sklearn
  memory: 100Ki
"""

simple_lr_model = yaml.safe_load(simple_lr_model)

In [ ]:
from  seldon_deploy_sdk import ModelsApi

# create an instance of the API class
models_api = ModelsApi(api_client)
models_api.create_model(namespace=NAMESPACE, model=simple_lr_model)

In [ ]:
# models_api.delete_model(namespace=NAMESPACE, name=MODEL_NAME)

In [ ]:
PIPELINE_NAME = "simple-lr-pipeline2"

simple_lr_pipeline = f"""
apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: {PIPELINE_NAME}
  namespace: {NAMESPACE}
spec:
  steps:
    - name: {MODEL_NAME}
  output:
    steps:
    - {MODEL_NAME}
"""

simple_lr_pipeline = yaml.safe_load(simple_lr_pipeline)

In [ ]:
from seldon_deploy_sdk import PipelinesApi

# create an instance of the API class
pipelines_api = PipelinesApi(api_client)
pipelines_api.create_pipeline(namespace=NAMESPACE, mldeployment=simple_lr_pipeline)

Check the status of the pipeline deployment

In [ ]:
pipelines_api.read_pipeline(name=PIPELINE_NAME, namespace=NAMESPACE).status.conditions

Run an example request to the pipeline.  Note that the endpoint does not require a namespace and a model.  However, two headers must be included in the request:
* Seldon-Model: [pipeline-name].pipeline
* Host: seldon-gitops.inference.seldon

In [ ]:
endpoint = f"http://{SD_IP}/v2/models/{PIPELINE_NAME}/infer"
endpoint

In [ ]:
import requests
import json

request = {"inputs": [
    {"name": "input-1",
    "datatype": 
    "INT32",
    "shape": [2, 2],
    "data": [
        [4, 4],
        [7, 7]]
        }]}

headers = {
    "Seldon-Model": f"{PIPELINE_NAME}.pipeline", 
    "Host": f"{NAMESPACE}.inference.seldon"
    }

response = requests.post(endpoint, json=request, headers=headers)
print(json.dumps(response.json(), indent=2))

### Train and deploy a canary model

Train a simple xgboost model and serialize it as json 

In [ ]:
import xgboost as xgb

# Create the xgboost model
model = xgb.XGBRegressor()

# Train the model using the training data
model.fit(X, y)

model.predict([[1, 1], [5, 5]])

model.save_model('simple-xgb.json')

In [ ]:
!gsutil cp simple-xgb.json gs://andrew-seldon/sdk-examples/simple-xgb/model.json

Add a model settings file, as required by MLServer

In [ ]:
%%writefile model-settings.json
{
    "name": "simple-xgb-model",
    "implementation": "mlserver_xgboost.XGBoostModel"
}

In [ ]:
!gsutil cp model-settings.json gs://andrew-seldon/sdk-examples/simple-xgb/model-settings.json

Deploy the model

In [ ]:
CANARY_MODEL_NAME = "simple-xgb-model2"
CANARY_URI = "gs://andrew-seldon/sdk-examples/simple-xgb"

# deploy xgb model
simple_xgb_model = f"""
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: {CANARY_MODEL_NAME}
  namespace: {NAMESPACE}
spec:
  storageUri: {CANARY_URI}
  requirements:
  - xgboost
  memory: 100Ki
"""

simple_xgb_model = yaml.safe_load(simple_xgb_model)

models_api.create_model(namespace=NAMESPACE, model=simple_xgb_model)

Deploy the pipeline

__IMPORTANT:__ You must define two labels on the canary `pipeline`:
* seldon.io/pipeline: {PIPELINE_NAME}
* seldon.io/experiment: canary

In [ ]:
CANARY_PIPELINE_NAME = "simple-lr-pipeline2-canary"


simple_xgb_pipeline = f"""
apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: {CANARY_PIPELINE_NAME}
  namespace: {NAMESPACE}
  labels:
    seldon.io/pipeline: {PIPELINE_NAME}
    seldon.io/experiment: canary
spec:
  steps:
    - name: {CANARY_MODEL_NAME}
  output:
    steps:
    - {CANARY_MODEL_NAME}
"""

simple_xgb_pipeline = yaml.safe_load(simple_xgb_pipeline)

pipelines_api.create_pipeline(namespace=NAMESPACE, mldeployment=simple_xgb_pipeline)

Create the experiment using the SDK
__CAVEATS:__
* The `create_canary_experiment` endpoint automatically creates the `experiment` without you having to explicitly define it -- it does this because of the labels defined in the canary `pipeline`
* There is currently a bug in the UI where you won't be able to see the canary deployment, however, the `experiment` _has_ been created.  We are working on a fix for this.

In [ ]:
from seldon_deploy_sdk import ExperimentsApi

# create an instance of the API class
experiments_api = ExperimentsApi(api_client)
experiments_api.create_canary_experiment(name=PIPELINE_NAME, namespace=NAMESPACE, weight=50)

### Model Metadata Catalog Examples

In [ ]:
from seldon_deploy_sdk import ModelMetadataServiceApi, V1Model
from seldon_deploy_sdk.rest import ApiException

metadata_api = ModelMetadataServiceApi(api_client)

##### Add single model to the Model Catalogue

In [ ]:
model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-test-model-alpha-repo",
        "an arbitrary tag": "true",
    },
)
try:
    # Create a Model Metadata entry.
    api_response = metadata_api.model_metadata_service_create_model_metadata(model)
except ApiException as e:
    print(f"Couldn't create model: {json.loads(e.body)['message']}")

##### Add multiple models to the Model Catalogue

In [ ]:
models = [
    #     Same model different versions
    {
        "uri": "gs://test-model-beta-v1.0.0",
        "name": "beta",
        "version": "v1.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Jon"},
    },
    {
        "uri": "gs://test-model-beta-v2.0.0",
        "name": "beta",
        "version": "v2.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
    {
        "uri": "gs://test-model-beta-v3.0.0",
        "name": "beta",
        "version": "v3.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
]

for model in models:
    body = V1Model(**model)
    try:
        api_response = metadata_api.model_metadata_service_create_model_metadata(body)
    except ApiException as e:
        print(f"Couldn't create model: {json.loads(e.body)['message']}")


##### List all models in the Model Catalogue

In [ ]:
try:
    # List Model Metadata entries.
    api_response = metadata_api.model_metadata_service_list_model_metadata()
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

##### Get all version of a given model (named "beta")

In [ ]:
try:
    # List Model Metadata entries.
    api_response = metadata_api.model_metadata_service_list_model_metadata(name="beta", tags={"author": "Jon"})
    print("Filter by name=beta")
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

# uri = 'uri_example'
# name = 'name_example'
# version = 'version_example'
# artifactType = 'artifactType_example'
# task_type = 'task_type_example'
# model_type = 'model_type_example'

##### Get all models authored by Bob (tags.author = Bob)

In [ ]:
try:
    # List Model Metadata entries.
    api_response = metadata_api.model_metadata_service_list_model_metadata(tags={"author": "Bob"})
    print("Filter by name=beta")
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

##### Modify model metadata entry in the Model Catalogue

In [ ]:
try:
    # Get Model Metadata entries.
    api_response = metadata_api.model_metadata_service_list_model_metadata(uri="gs://test-model-alpha-v1.0.0")
    print("Before update:")
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")


model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-other-repo",
        "an arbitrary tag": "true",
        "an additional tag": "123",
    },
)

try:
    # Update a Model Metadata entry.
    api_response = metadata_api.model_metadata_service_update_model_metadata(model)
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")
    
try:
    # List Model Metadata entries.
    api_response = metadata_api.model_metadata_service_list_model_metadata(
        uri="gs://test-model-alpha-v1.0.0"
    )
    print("After update:")
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

##### Get runtime information for a model

In [ ]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = metadata_api.model_metadata_service_list_runtime_metadata_for_model(
        model_uri=URI, 
        deployment_status="Running"
    )
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")


##### Get model information for a deployment

In [ ]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = metadata_api.model_metadata_service_list_runtime_metadata_for_model(
        deployment_name=DEPLOYMENT_NAME, 
        deployment_namespace=NAMESPACE
    )
    print(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")